In [1]:
from hamiltonian import HamiltonianLB, HamiltonianH2, HamiltonianWA

In [2]:
lih = HamiltonianLB('LiH', 1.5)
beh2 = HamiltonianLB('BeH2', 1.3)

h2_jw = HamiltonianH2('jw')
h2_parity = HamiltonianH2('parity')
h2_bk = HamiltonianH2('bk')

water_jw = HamiltonianWA('water', 'jw')
water_parity = HamiltonianWA('water', 'parity')
water_bk = HamiltonianWA('water', 'bk')

ammonia_jw = HamiltonianWA('ammonia', 'jw')
ammonia_parity = HamiltonianWA('ammonia', 'parity')
ammonia_bk = HamiltonianWA('ammonia', 'bk')

In [3]:
%time lih.pauli_rep.ground()
%time beh2.pauli_rep.ground()
%time h2_jw.pauli_rep.ground()
%time water_jw.pauli_rep.ground()
%time energy, state = ammonia_jw.pauli_rep.ground()

CPU times: user 116 ms, sys: 3.5 ms, total: 119 ms
Wall time: 116 ms
CPU times: user 268 ms, sys: 1.88 ms, total: 270 ms
Wall time: 269 ms
CPU times: user 548 ms, sys: 38.6 ms, total: 586 ms
Wall time: 422 ms
CPU times: user 22.9 s, sys: 2.48 s, total: 25.4 s
Wall time: 10.1 s
CPU times: user 7min 27s, sys: 2min 13s, total: 9min 41s
Wall time: 4min 38s


In [6]:
from sparse import energy as sparseenergy
from sparse import energy_kron

pr = beh2.pauli_rep
energy, state = pr.ground()
print("energy: ", energy)
%time energy_sparse = sparseenergy(pr, state)
print("energy_sparse: ", energy_sparse)
%time energy_kron = energy_kron(pr, state)
print("energy_kron: ", energy_kron)

energy:  -3.989368880501239
CPU times: user 254 ms, sys: 1.71 ms, total: 256 ms
Wall time: 255 ms
energy_sparse:  -3.9893688805012424
CPU times: user 21.8 s, sys: 12 ms, total: 21.8 s
Wall time: 21.8 s
energy_kron:  -3.989368880501239


In [7]:
β = pr.local_dists_uniform()
%time pr.variance_local(energy, state, β)

CPU times: user 346 ms, sys: 1.5 ms, total: 347 ms
Wall time: 346 ms


18.179850794404405

In [10]:
print("ell_1: ", pr.variance_ell_1(energy))

β = pr.local_dists_pnorm(1)
print("1_norm: ", pr.variance_local(energy, state, β))

β = pr.local_dists_pnorm(2)
print("2_norm: ", pr.variance_local(energy, state, β))

β = pr.local_dists_pnorm('infinity')
print("max_norm: ", pr.variance_local(energy, state, β))

ell_1:  42.28949301890359
1_norm:  88.57935992546827
2_norm:  80.69169432668934
max_norm:  79.62377675669384


# Variance optimisation (old and wrong, but convex)

If we use a uniform/biased pauli/magic bases algorithm to estimate the energy, and we assume that the state is a product-state (or close to a produce-state) then we end up wanting to minimise:
$$
    \Var[\nu]
    =
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    +
    \sum_{\Qarrow,\Rarrow \in \{I,Z\}^{\otimes n}} \alpha_\Qarrow \alpha_\Rarrow \prod_{i} m_i(1-\delta_{Q_i R_i})
    - \tr(H_0\rho)^2
$$
(Above is the chemistry HF situation, where the product-state is in the $Z$-basis, but this statement wouldn't depend on using $Z$-basis uniformly.)

We end up wanting to minimise:
$$
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    \qquad
    \textrm{subject to}
    \qquad
    \beta_{i,X}+\beta_{i,Y}+\beta_{i,Z}=1 \,\forall i,
    \qquad
    \beta_{i,P}\ge 0
$$

In [11]:
β_optimal = pr.local_dists_optimal()
print(pr.variance_local(energy, state, β_optimal))

13.845088491932902


In [12]:
def variances_dict(pauli_rep):
    pr = pauli_rep
    dic = {}
    
    energy, state = pr.ground()
    print("energy :", energy)

    # ell_1
    var = pr.variance_ell_1(energy)
    print("ell 1: ", var)
    dic['ell_1'] = var
    
    # uniform
    β = pr.local_dists_uniform()
    var = pr.variance_local(energy, state, β)
    print("uniform: ", var)
    dic['uniform'] = var
    
    # optimal
    β = pr.local_dists_optimal()
    var = pr.variance_local(energy, state, β)
    print("optimal: ", var)
    dic['optimal'] = var
    
    return dic

In [13]:
variances_ALL = {}
beta_optimal_ALL = {}

In [14]:
name = 'lih'
pr = lih.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.1014068450599004
ell 1:  8.798891105500466
uniform:  2.230730161038921
optimal:  1.576299391494913
CPU times: user 875 ms, sys: 77 ms, total: 952 ms
Wall time: 368 ms


In [15]:
name = 'beh2'
pr = beh2.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -3.989368880501248
ell 1:  42.28949301890355
uniform:  18.179850794404423
optimal:  13.845088491932891
CPU times: user 4.99 s, sys: 490 ms, total: 5.48 s
Wall time: 1.95 s


In [16]:
name = 'h2_jw'
pr = h2_jw.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.8608605555207598
ell 1:  119.6790600190591
uniform:  51.3998202138761
optimal:  17.742071936811435
CPU times: user 10.3 s, sys: 1.03 s, total: 11.3 s
Wall time: 3.21 s


In [17]:
name = 'h2_parity'
pr = h2_parity.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

/opt/anaconda3/envs/qiskit18/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


energy : -1.8608605555207567
ell 1:  119.6790600190592
uniform:  70.76147441962259
optimal:  18.924750112035007
CPU times: user 34.6 s, sys: 3.92 s, total: 38.5 s
Wall time: 8.64 s


In [18]:
name = 'h2_bk'
pr = h2_bk.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.8608605555207567
ell 1:  119.67906001905945
uniform:  168.86600324405416
optimal:  19.516932420931475
CPU times: user 20.4 s, sys: 2.18 s, total: 22.5 s
Wall time: 5.34 s


In [19]:
pr = water_jw.pauli_rep
%time β = pr.local_dists_optimal()
β

CPU times: user 6min 45s, sys: 1min 7s, total: 7min 53s
Wall time: 59.8 s


{0: [0.43257858556565915, 0.43257858722135467, 0.1348428272129862],
 1: [0.24098007537433425, 0.2409800708577077, 0.518039853767958],
 2: [0.13870541903721406, 0.13870542588015417, 0.7225891550826317],
 3: [0.13957872152763764, 0.13957871134125624, 0.7208425671311062],
 4: [0.16984472160573807, 0.1698447243114196, 0.6603105540828423],
 5: [0.2675208449775755, 0.2675208434379076, 0.464958311584517],
 6: [0.11647067744295292, 0.11647067819430176, 0.7670586443627454],
 7: [0.43257857708372555, 0.43257860312684154, 0.13484281978943294],
 8: [0.24098005034367995, 0.24098008250406633, 0.5180398671522537],
 9: [0.13870542248842968, 0.13870542724539398, 0.7225891502661764],
 10: [0.13957872657623138, 0.13957872195005588, 0.7208425514737127],
 11: [0.16984475333796817, 0.16984473040786618, 0.6603105162541657],
 12: [0.2675208758292949, 0.26752084565650264, 0.46495827851420257],
 13: [0.11647066971784335, 0.1164706702788042, 0.7670586600033525]}

In [20]:
%time energy, state = pr.ground()

CPU times: user 21.7 s, sys: 2.29 s, total: 24 s
Wall time: 9.77 s


In [21]:
%time var = pr.variance_ell_1(energy)
print("ell 1: ", var)

CPU times: user 13 µs, sys: 1e+03 ns, total: 14 µs
Wall time: 18.1 µs
ell 1:  4363.497773126066


In [22]:
%time var = pr.variance_local(energy, state, pr.local_dists_uniform())
print("uniform: ", var)

CPU times: user 2min 55s, sys: 1.6 s, total: 2min 57s
Wall time: 2min 57s
uniform:  2839.0394682188467


In [23]:
%time var = pr.variance_local(energy, state, β)
print("optimal: ", var)

CPU times: user 2min 54s, sys: 1.54 s, total: 2min 56s
Wall time: 2min 56s
optimal:  257.54509937269825
